
# **Media Prediction Data Construction**

**Media/Culture Information**

1.1 MediaDive | Provides media IDs, components/annotations, associated taxa, and other relevant information

1.2 BacDive | Provides culture/isolation info, metabolic annotations (including EC and metabolites), and more (1.1)

**Taxa to EC**

2.1 UniProtKB (species) | Queries UniProtKB for ec numbers from an input of species names (1.1)

2.2 UniProtKB (taxon_id) | Queries UniProtKB for ec numbers from an input of NCBI taxon_id's (1.2)

2.3 Final taxa2ec dataframe | Formatting identifiers and associated ec numbers (1.2, 2.1, 2.3)

**Media to EC**

3.1 KEGG (KEGG CPD) | Queries KEGG for ec numbers from an input of KEGG compounds (1.1)


**EC to Cofactor**

4.1 Subsetting data based on isolation information

4.2 UniProtKB (ec) | Queries UniProtKB for cofactor information from an input of ec numbers (2.3, 3.1)


**EC to KEGG Ortholog**

5.1 KEGG (ec) | Queries KEGG for KO numbers from a list of EC numbers


**Metabolites**

6.1 Formatting metabolite information from BacDive (1.2)


**Environmental Parameters**


**Concentration Mapping**


*Search terms: "final dataframe", "#SAVE", ...*

In [1]:
import pandas as pd
from tqdm import tqdm
from io import StringIO
import ast

import sys
sys.path.append("..")
import os
DATA_DIR = 'C:/Users/jakel/Desktop/code/data/'
#DATA_DIR should have sub-directories "bacdive", "cofactors", "concentrations", "kegg", "MAGs", "mediadive", and "uniprot"

import requests
import re
from requests.adapters import HTTPAdapter, Retry

## Media and Associated Taxa

### [1.1] MediaDive

Returns all information (media id's, components, component id's, characteristics, associated taxa information, etc.) used for subsequent analyses.

In [2]:
import modules.mediadive as md

Retrieve all MediaDive info

In [3]:
# Retrieve all available media from MediaDive
md_media_df = md.get_media()
md_media_df.to_csv(os.path.join(DATA_DIR, "mediadive", "mediadive-media.csv"), index=False) #SAVE

In [4]:
# Create media_id_list
md_media_df = md_media_df.rename(columns={"id": "media_id"})
media_id_list = md_media_df["media_id"].astype(str).unique()

# Use media_id_list to retrieve media composition information
md_comp_df = md.get_composition(media_id_list)

# Use media_id_list to retrieve media-associated strain information
md_strains_df = md.get_strains(media_id_list)

md_comp_df.to_csv(os.path.join(DATA_DIR, "mediadive","mediadive-media-comp.csv"), sep=";", index=False) #SAVE
md_strains_df.to_csv(os.path.join(DATA_DIR, "mediadive", "mediadive-media-strains.csv"), sep=";", index=False) #SAVE

100%|██████████| 3322/3322 [08:10<00:00,  6.77it/s]


Merge MediaDive outputs

In [5]:
# Merge media composition and strains info
md_df = pd.merge(left=md_comp_df, right=md_strains_df, on="media_id", how="outer", indicator=True)

# Merge media information with original dataframe
md_df = pd.merge(left=md_media_df, right=md_df, on="media_id", how="left", indicator=False)

# Add extra column indicating the source of the data
md_df = md_df.rename(columns={"_merge": "merge_source"})
md_df["merge_source"] = md_df["merge_source"].cat.rename_categories({"right_only": "composition_only", "left_only": "strains_only"})

md_df.to_csv(os.path.join(DATA_DIR,"mediadive","mediadive-all.csv"), sep=";", index=False) #SAVE

**MediaDive final dataframe**

In [6]:
md_df = pd.read_csv(os.path.join(DATA_DIR,"mediadive","mediadive-all.csv"), sep = ';')
md_df.head()

,media_id,name,complex_medium,source,link,min_pH,max_pH,reference,description,components,component_ids,strain_id,species,ccno,bacdive_id,merge_source
0,1,NUTRIENT AGAR,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",29.0,Comamonas testosteroni,DSM 38,2912.0,both
2,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",30.0,Delftia acidovorans,DSM 39,2941.0,both
3,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",39.0,Acidovorax delafieldii,DSM 64,2885.0,both
4,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",52.0,Pseudomonas putida,DSM 84,12895.0,both


### [1.2] BacDive

Returns ec numbers, environmental info, metabolite data, taxonomic information, culturing info, and more from an input of bacdive_id's

In [7]:
import bacdive
import modules.bacdive as bd

Initialize BacDive, prepare IDs

In [8]:
# Retrieve bacdive_id's from the 'md_df' MediaDive table
md_df = pd.read_csv(os.path.join(DATA_DIR, "mediadive", "mediadive-all.csv"), sep=";")
bd_id_list = md_df["bacdive_id"].dropna().astype(int).astype(str).unique()
len(bd_id_list)

13736

Retrieval of BacDive info

In [9]:
# Initialize client
client = bacdive.BacdiveClient('wjlowe03@gmail.com', 'UNINA_Bacdive')

step = 100
bacdive_df = []

for idx_start in tqdm(range(0, len(bd_id_list), step)): #status bar
    id_list = ";".join(bd_id_list[idx_start:idx_start+step]) 
    bacdive_df.append(bd.taxon2ec(id_list=id_list, client=client))

bacdive_df = pd.concat(bacdive_df, axis=0, ignore_index=True)
bacdive_df = bacdive_df.drop("reference", axis=1)

bacdive_df.to_csv(os.path.join(DATA_DIR, "bacdive", "bacdive-all.csv"), index=False) #SAVE

-- Authentication successful --


100%|██████████| 138/138 [03:27<00:00,  1.51s/it]


**BacDive final dataframe**

In [10]:
bacdive_df = pd.read_csv(os.path.join(DATA_DIR, 'bacdive', 'bacdive-all.csv'), low_memory=False)
bacdive_df.head()

,general_@ref,bacdive_id,dsmz_id,general_keywords,general_description,taxon_id,ncbi_tax_id_matching_level,strain_history_@ref,strain_history_history,general_doi,...,api_id32sta_beta_gur,physiology_and_metabolism_murein,physiology_and_metabolism_api_list,isolation_enrichment_culture,isolation_enrichment_culture_temperature,multicellular_morphology_complex_color,api_list_beta_hem,metabolite_tests_citrate_test,compound_production_excreted,isolation_enrichment_culture_duration
0,21113,24370,11532.0,"['Bacteria', 'human pathogen']",Pseudomonas fluorescens PWD34 is a human patho...,294,species,21113.0,"<- W. Duetz, RIVM, Bilthoven; PWD34",10.13145/bacdive24370.20240510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21111,24368,304.0,Bacteria,Pseudomonas sp. DSM 304 is a bacterium of the ...,306,species,21111.0,"<- IMG, 1591 (<i>P. fluorescens</i>) <- H. Stolp",10.13145/bacdive24368.20240510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20542,23995,30059.0,Bacteria,Lelliottia amnigena 21824 is a bacterium that ...,61646,species,20542.0,<- Bakteriologisches Institut der Sueddeutsche...,10.13145/bacdive23995.20240510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1479,17621,3849.0,"['16S sequence', 'Bacteria', 'plant pathogen']",Xanthomonas citri subsp. malvacearum XM13 is a...,346,species,NaN,NaN,10.13145/bacdive17621.20240510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1480,17596,3850.0,"['genome sequence', 'Bacteria', 'obligate aero...",Xanthomonas campestris DSM 3850 is an obligate...,339,species,NaN,NaN,10.13145/bacdive17596.20240510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Taxa to EC

### [2.1] UniProtKB taxa2ec (species name)

Returns ec numbers and identifier information from UniProtKB for an input of species (MediaDive)

In [11]:
import modules.uniprot as uni

Format species name from the 'md_df' MediaDive dataframe for querying UniProtKB

In [12]:
species_df = md_df.copy()
species_df['species'] = species_df['species'].replace(' ','+', regex=True)
species_list = set(species_df['species'].to_list())
len(species_list)

12405

Retrieval of UniProtKB info

In [13]:
# Import and run function to retrieve ec's associated with each species
    # Note: currently, script only checks reviewed entries (SwissProt), and ignores 1000's of tREMBL entries
    # Errors represent where species are totally absent, whereas non-reviewed entries show as 'x species with no data'
    
uniprot_df = uni.species2ec(species_list)
uniprot_df.to_csv(os.path.join(DATA_DIR, "uniprot", "uniprot-all.csv"), index=False) #SAVE

Processing species:  20%|█▉        | 2480/12405 [05:36<18:17,  9.04it/s]  

HTTP error occurred for Protocrea+illino&euml;nsis: 400 Client Error: Bad Request for url: https://rest.uniprot.org/uniprotkb/search?fields=accession%2Cec%2Corganism_name%2Corganism_id%2Ccc_cofactor%2Cid&format=tsv&size=500&query=%28organism_name%3AProtocrea+illino&euml;nsis%29+AND+%28ec%3A*%29+AND+%28reviewed%3Atrue%29


Processing species:  23%|██▎       | 2803/12405 [06:16<23:56,  6.68it/s]

HTTP error occurred for Sphingobacterium+composti+[homonym]: 400 Client Error: Bad Request for url: https://rest.uniprot.org/uniprotkb/search?fields=accession%2Cec%2Corganism_name%2Corganism_id%2Ccc_cofactor%2Cid&format=tsv&size=500&query=%28organism_name%3ASphingobacterium+composti+%5Bhomonym%5D%29+AND+%28ec%3A*%29+AND+%28reviewed%3Atrue%29


Processing species: 100%|██████████| 12405/12405 [27:33<00:00,  7.50it/s] 


10670 species with no data


**UniProtKB (MediaDive species name) final dataframe**

In [14]:
uniprot_df = pd.read_csv(os.path.join(DATA_DIR, "uniprot", "uniprot-all.csv"))

### [2.2] UniProtKB taxa2ec (taxon_id)

Combined approach: uses NCBI taxon_id's (species level) from BacDive to query UniProtKB

In [27]:
bacdive_df = pd.read_csv(os.path.join(DATA_DIR, "bacdive", "bacdive-all.csv"), low_memory=False)

# Multiple taxon_id's assigned for the same species level -- can explode or remove here
bacdive_df['taxon_id'] = bacdive_df['taxon_id'].astype(str)
bacdive_df['taxon_id'] = bacdive_df['taxon_id'].str.split(';').explode('taxon_id')

# Filter nan and/or joined IDs
nan = bacdive_df['taxon_id'].str.contains("nan")
#joined = bacdive_df['taxon_id'].str.contains(";")
bacdive_df = bacdive_df[~nan]

print(len(bacdive_df), "species-level taxon_ids")

13624 species-level taxon_ids


In [28]:
tax_ids = bacdive_df['taxon_id'].to_list()

ncbi_df = uni.taxon2ec(tax_ids)
ncbi_df['ec_uniprot'] = ncbi_df['ec_uniprot'].str.split('; ').explode('ec_uniprot')

ncbi_df.to_csv(os.path.join(DATA_DIR, 'bacdive', 'ncbi-ec.csv'), index=False) #SAVE

Processing species: 100%|██████████| 13624/13624 [1:41:12<00:00,  2.24it/s]  


3418 species with no data


**UniProtKB (NCBI taxon IDs) final dataframe**

In [29]:
ncbi_df = pd.read_csv(os.path.join(DATA_DIR, 'bacdive', 'ncbi-ec.csv'))
ncbi_df.head()

,species,ec_uniprot
0,294,4.1.2.61
1,294,1.14.13.2
2,294,1.-.-.-
3,294,3.1.1.2
4,294,1.14.13.-


### [2.3] Formatting final taxa2ec table

Reformat outputs from [1.2-2.2]

In [30]:
bacdive_df = pd.read_csv(os.path.join(DATA_DIR, 'bacdive', 'bacdive-all.csv'), low_memory=False)
uniprot_df = pd.read_csv(os.path.join(DATA_DIR, 'uniprot', 'uniprot-all.csv'))
ncbi_df = pd.read_csv(os.path.join(DATA_DIR, 'bacdive', 'ncbi-ec.csv'))

# BacDive taxa2ec (grouped by bacdive_id)
bacdive_ec = bacdive_df[['bacdive_id','taxon_id','type_strain','ec']].copy()
bacdive_ec['ec'] = bacdive_ec['ec'].str.replace("'", "")
bacdive_ec = bacdive_ec.rename(columns={'ec': 'ec_bacdive'})

# UniProtKB taxa2ec (grouped by species name)
uniprot_ec = uniprot_df.copy()
uniprot_ec['species'] = uniprot_ec['species'].replace(r'\+',' ', regex=True)
uniprot_ec['ec_uniprot'] = uniprot_ec['ec_uniprot'].str.replace(";", ",")
uniprot_ec = uniprot_ec.groupby("species", as_index=False)["ec_uniprot"].apply(lambda x: "[%s]" % ', '.join(x))

# NCBI taxa2ec (grouped by taxon_id)
ncbi_ec = ncbi_df.astype(str).copy()
ncbi_ec = ncbi_ec.rename(columns={'species': 'taxon_id', 'ec_uniprot': 'ec_ncbi'})
ncbi_ec = ncbi_ec.groupby("taxon_id", as_index=False)["ec_ncbi"].apply(lambda x: "[%s]" % ', '.join(x))

Merge 'md_df' MediaDive dataframe with formatted outputs

In [31]:
media_df = md_df.copy()

# Completing merge in multiple steps since we're merging on different columns
merged1 = pd.merge(left = media_df, right = uniprot_ec, on = 'species', how = 'left')
merged2 = pd.merge(left = merged1, right = bacdive_ec, on = 'bacdive_id', how = 'left')
merged3 = pd.merge(left = merged2, right = ncbi_ec, on = 'taxon_id', how = 'left')

merged3.to_csv(os.path.join(DATA_DIR, "taxa2ec-explode.csv"), index=False) #SAVE
merged3.head()

,media_id,name,complex_medium,source,link,min_pH,max_pH,reference,description,components,...,strain_id,species,ccno,bacdive_id,merge_source,ec_uniprot,taxon_id,type_strain,ec_bacdive,ec_ncbi
0,1,NUTRIENT AGAR,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...",...,29.0,Comamonas testosteroni,DSM 38,2912.0,both,"[2.6.1.1, 4.1.1.12, 1.13.11.74, 1.13.11.76, 1....",285,no,"[1.9.3.1, 3.2.1.21, 3.5.1.5, 3.5.3.6]","[4.3.99.3, 5.4.2.-, 7.2.1.1, 3.1.26.4, 5.3.1.1..."
2,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...",...,30.0,Delftia acidovorans,DSM 39,2941.0,both,"[1.14.11.43, 1.14.11.44, 3.1.1.75, 6.3.4.2, 5....",80866,yes,"[1.9.3.1, 3.2.1.21, 3.5.1.5, 3.5.3.6]","[2.1.2.10, 2.7.9.2, 2.1.1.185, 1.1.1.3, 7.1.1...."
3,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...",...,39.0,Acidovorax delafieldii,DSM 64,2885.0,both,[1.1.1.37],47920,yes,"[3.2.1.51, 3.2.1.24, 3.2.1.52, 3.2.1.21, 3.2.1...","[1.3.1.76, 2.1.1.107, 4.99.1.4, 1.1.1.35, 4.2...."
4,1a,REACTIVATION WITH LIQUID MEDIUM 1,1,DSMZ,https://www.dsmz.de/microorganisms/medium/pdf/...,7.0,7.0,NaN,NaN,"['Peptone', 'Meat extract', 'Agar', 'Distilled...",...,52.0,Pseudomonas putida,DSM 84,12895.0,both,"[5.1.1.10, 5.3.3.1, 1.18.1.3, 1.2.98.1, 1.18.1...",303,no,"[3.2.1.21, 3.5.1.5, 3.5.3.6]","[6.3.2.13, 3.4.24.-, 6.3.4.16, 6.3.5.5, 2.1.2...."


**taxa2ec final dataframe**

In [32]:
final_df = merged3[["media_id", "species", "taxon_id", "ec_uniprot", "ec_bacdive","ec_ncbi"]].copy()

# Melt ec columns and attribute ec source
final_df = final_df.melt(
    id_vars=["media_id", "species", "taxon_id"],
    value_vars=["ec_uniprot", "ec_bacdive", "ec_ncbi"],
    value_name="ec",
    var_name="source"
)

# Format source and ec columns
final_df["source"] = final_df["source"].str.replace("ec_", "")
final_df['ec'] = final_df['ec'].astype(str).copy()
final_df["ec"] = final_df["ec"].str.replace("[", "").str.replace("]", "")
final_df['ec'] = final_df['ec'].str.split(', ')
final_df = final_df.explode('ec')

# Remove rows with nan 'ec' values
final_df = final_df.copy()
substring = 'nan'
filter = final_df['ec'].str.contains(substring) # create filter
final_df = final_df[~filter]

final_df.to_csv(os.path.join(DATA_DIR, "taxa2ec-final.csv"), index=False) #SAVE
final_df

,media_id,species,taxon_id,source,ec
1,1a,Comamonas testosteroni,285,uniprot,2.6.1.1
1,1a,Comamonas testosteroni,285,uniprot,4.1.1.12
1,1a,Comamonas testosteroni,285,uniprot,1.13.11.74
1,1a,Comamonas testosteroni,285,uniprot,1.13.11.76
1,1a,Comamonas testosteroni,285,uniprot,1.14.13.23
...,...,...,...,...,...
92641,J1328,Mariniphaga anaerophila,1484053,ncbi,2.1.2.11
92641,J1328,Mariniphaga anaerophila,1484053,ncbi,2.3.1.31
92641,J1328,Mariniphaga anaerophila,1484053,ncbi,1.1.1.205
92641,J1328,Mariniphaga anaerophila,1484053,ncbi,4.1.1.49


## Media to EC

### [3.1] KEGG media2ec (CPD)

Returns ec numbers from KEGG for an input of CPDs (MediaDive)

In [4]:
from Bio.KEGG import REST
import modules.mediadive as md
import modules.kegg as kegg

Retrieve KEGG and ChEBI compound IDs from an input of MediaDive 'component_ids'

In [5]:
# Retrieve 'component_ids' from the 'md_df' MediaDive dataframe
md_df = pd.read_csv(os.path.join(DATA_DIR,"mediadive","mediadive-all.csv"), sep = ';')

comps_df = md_df[['media_id','components','component_ids']].copy()
comps_df['media_id'] = comps_df['media_id'].drop_duplicates()
comps_df = comps_df.dropna()

# Extract the component_ids into a list
comps_df['component_ids'] = comps_df['component_ids'].astype(str)

def extract_ids(comps_df, component_ids):
    id_set = set()  # Use a set to avoid duplicate IDs
    for ids in comps_df['component_ids']:
        id_list = eval(ids)  # Convert the string representation of the list to an actual list
        id_set.update(id_list)
    return list(id_set)

# Extract IDs
id_list = extract_ids(comps_df, 'ids')

Retrieve associated ECs for each compound

In [6]:
# Retrieve compound IDs from component IDs (MediaDive)
compound_df = md.get_compounds(id_list)

# Making 'cpd_list' using the KEGG compound IDs
cpd_df = compound_df['KEGG cpd'].dropna().copy()
cpd_list = cpd_df.to_list()

# Retrieve ECs from CPDs
cpd2ec_df = kegg.compound2ec(cpd_list) # HTTP errors = no ECs associated with this compound on KEGG

# Merge dataframes with component_id, compound, and ec information
cpd_merge = pd.merge(left=compound_df, right=cpd2ec_df, on="KEGG cpd", how="outer")
cpd_merge = cpd_merge.drop_duplicates()

cpd_merge.to_csv(os.path.join(DATA_DIR,"kegg","kegg-compound-ec.csv"), index=False) #SAVE

100%|██████████| 769/769 [01:56<00:00,  6.59it/s]


HTTP error occurred for C00293: 404 Client Error: Not Found for url: https://rest.kegg.jp/get/compound:C00293
HTTP error occurred for C00382: 404 Client Error: Not Found for url: https://rest.kegg.jp/get/compound:C00382


Format final table for media2ec

**media2ec final dataframe**

In [7]:
# Explode md_df on component_id
component_df = md_df.copy()
component_df['component_ids'] = md_df['component_ids'].str.strip('[]')
component_df['component_ids'] = component_df['component_ids'].str.split(', ').explode('component_ids')
component_df = component_df.rename(columns={'component_ids': 'component_id'})

# Merge md_df with media2ec (KEGG), remove NaN 'KEGG cpd' values
cpd_merge['component_id'] = cpd_merge['component_id'].astype(str)
comp_comp = pd.merge(left=component_df, right=cpd_merge, on='component_id', how='outer')
kegg2ec = comp_comp.loc[comp_comp['KEGG cpd'].notnull()].copy()
kegg2ec = kegg2ec.loc[kegg2ec['Enzyme'].notnull()]

#NOTE: not all 'component id's' have an associated 'KEGG compound', and not all 'KEGG compound's' have an associated 'Enzyme'...this means our final table has a loss of data from our original input
    #Can try to remedy the data loss by splitting complex media components into their simpler forms

# Load and format taxon_id's from bacdive dataframe
taxon_df = pd.read_csv(os.path.join(DATA_DIR,"bacdive","bacdive-all.csv"), low_memory=False)
taxon_df = taxon_df[['bacdive_id','taxon_id']]

# Merge taxon_id's to kegg2ec table
media_final = pd.merge(left=kegg2ec, right=taxon_df, on='bacdive_id', how='outer')

media_final = media_final[['media_id','taxon_id','component_id','KEGG cpd','Enzyme']].copy()
media_final = media_final.dropna(subset='Enzyme')

media_final.to_csv(os.path.join(DATA_DIR, "media2ec-final.csv"), index=False) #SAVE
media_final.head()
    #Two different model inputs: one with component id's, one with enzymes

,media_id,taxon_id,component_id,KEGG cpd,Enzyme
4,J597,438,199,C00369,2.4.1.1 2.4.1.18 2.4.1.19 ...
12,J97,146475,4,C00001,1.1.1.1 1.1.1.22 1.1.1.23 ...
16,J710,104098,199,C00369,2.4.1.1 2.4.1.18 2.4.1.19 ...
19,J129,62140,46,C00120,1.-.-.- 2.8.1.6 3.5.1.12 ...
22,J97,110479,37,C12486,1.14.-.-


**media2ec final dataframe (exploded)**

In [8]:
media_final = pd.read_csv(os.path.join(DATA_DIR, "media2ec-final.csv"))

# Function to split the column by 6-9 spaces (variable delimitation...idk why)
def split_and_clean(value):
    # Split the string based on 6 to 9 spaces
    split_values = re.split(r'\s{6,9}', value)
    # Remove any remaining spaces from the split values
    split_values = [v.strip() for v in split_values]
    return split_values

# Split and Explode
media_final['split_column'] = media_final['Enzyme'].apply(split_and_clean).copy()
media_split = media_final.explode('split_column').reset_index(drop=True)
media_split = media_split.drop(columns=['Enzyme'])
media_split = media_split.rename(columns={'split_column': 'ec_KEGG'})

media_split['ec_KEGG'] = media_split['ec_KEGG'].dropna()
media_split.to_csv(os.path.join(DATA_DIR, "media2ec-explode.csv"), index=False) #SAVE
media_split.head()

,media_id,taxon_id,component_id,KEGG cpd,ec_KEGG
0,J597,438,199,C00369,2.4.1.1
1,J597,438,199,C00369,2.4.1.18
2,J597,438,199,C00369,2.4.1.19
3,J597,438,199,C00369,3.2.1.1
4,J97,146475,4,C00001,1.1.1.1


## EC to Cofactors

### [4.1] Subsetting data based on isolation information

Useful for selecting isolates from specific environments

In [9]:
bacdive_df = pd.read_csv('~/Desktop/code/data/bacdive/bacdive-all.csv', low_memory=False)
final_df = pd.read_csv('~/Desktop/code/data/taxa2ec-final.csv', low_memory=False)

# Retrieve isolation data from 'bacdive-all.csv'
iso_data = bacdive_df[['taxon_id', 'isolation,_sampling_and_environmental_information_isolation']].copy()
iso_data = iso_data.dropna()

# Example: filter for isolates from "thermal" sources
filtered_df = iso_data[iso_data['isolation,_sampling_and_environmental_information_isolation'].str.contains('thermal')]
#isolates = filtered_df['taxon_id'].to_list()

# Merge with info from 'taxa2ec-final.csv'
merge_iso = pd.merge(left=filtered_df, right=final_df, on='taxon_id', how='left')

merge_iso.to_csv(os.path.join(DATA_DIR, "isolate-ec.csv"), index=False) #SAVE
merge_iso.head()

,taxon_id,"isolation,_sampling_and_environmental_information_isolation",media_id,species,source,ec
0,"[{'NCBI tax id': 291995, 'Matching level': 'sp...","[{'@ref': 6972, 'sample type': 'hyperthermal c...",1a,Pseudomonas azotifigens,bacdive,1.11.1.6
1,"[{'NCBI tax id': 291995, 'Matching level': 'sp...","[{'@ref': 6972, 'sample type': 'hyperthermal c...",1a,Pseudomonas azotifigens,bacdive,1.9.3.1
2,"[{'NCBI tax id': 291995, 'Matching level': 'sp...","[{'@ref': 6972, 'sample type': 'hyperthermal c...",1a,Pseudomonas azotifigens,bacdive,3.2.1.23
3,"[{'NCBI tax id': 291995, 'Matching level': 'sp...","[{'@ref': 6972, 'sample type': 'hyperthermal c...",1a,Pseudomonas azotifigens,bacdive,1.1.1.1
4,"[{'NCBI tax id': 291995, 'Matching level': 'sp...","[{'@ref': 6972, 'sample type': 'hyperthermal c...",1a,Pseudomonas azotifigens,bacdive,3.4.21.50


### [4.2] UniProtKB ec2metals (EC)

Returns cofactor information for a list of ec numbers

In [2]:
import modules.cofactors as cofactors

Format list of ec numbers

In [3]:
# Load reference dataframe

df = pd.read_csv(os.path.join(DATA_DIR, 'taxa2ec-final.csv'), low_memory=False) #taxa2ec-final.csv media2ec-explode.csv isolate-ec.csv | compare inputs eventually

#final_ec_list = df['ec'].iloc[:50].to_list() #subset for first n
final_ec_list = df['ec'].to_list() #may have to change column name based on dataframe
final_ecs = set(final_ec_list) #remove duplicates in list

ec_list = list(final_ecs)

Retrieve cofactor information

In [4]:
# Retrieve cofactors
cofactor_df = cofactors.ec2metals(ec_list)

# Format ec2metals output
cofactors = cofactor_df.drop_duplicates().copy()
cofactors = cofactors.dropna(subset=['CofactorExtracted']).copy()
# List out all cofactors
CofactorFinal = cofactors.drop_duplicates(subset=['Query EC','CofactorFinal']).copy()
CofactorFinal.to_csv(os.path.join(DATA_DIR, "cofactors", "cofactors-subset.csv"), index=False) #SAVE

# Group cofactors by EC
grouped_cofactors = CofactorFinal.groupby("Query EC", as_index=False)["CofactorFinal"].apply(lambda x: "[%s]" % ', '.join(x))
grouped_cofactors.to_csv(os.path.join(DATA_DIR, "cofactors", "cofactors-grouped-subset.csv"), index=False) #SAVE
grouped_cofactors.head()

100%|██████████| 4090/4090 [1:11:26<00:00,  1.05s/it]


,Query EC,CofactorFinal
0,1.-.-.-,"[Fe, Zn, S, Mg, L-ascorbate, thiamine diphosph..."
1,1.1.-.-,"[Zn, Mg, FAD, Fe, S, pyrroloquinoline quinone,..."
2,1.1.1.-,"[Zn, Mg, metal, K, FAD, Fe, S, Ni, FMN, Mn, Cu..."
3,1.1.1.1,"[Zn, Fe]"
4,1.1.1.103,[Zn]


Format a table for machine learning

In [5]:
#ec_source = pd.read_csv(os.path.join(DATA_DIR, "media2ec-explode.csv"), low_memory=False) 
#ec_source = ec_source.rename(columns={"ec_KEGG": "ec"}) #for media2ec (KEGG)

#ec2cofactor_df = pd.read_csv(os.path.join(DATA_DIR, "cofactors", "cofactors-media2ec.csv"), low_memory=False) #CHANGE DEPENDING ON DESIRED MODEL
#ec2cofactor_df = ec2cofactor_df.rename(columns={"Query EC": "ec"})

#cofactor_model = pd.merge(left=ec_source, right=ec2cofactor_df, on="ec", how="left")
#cofactor_model = cofactor_model[["media_id","taxon_id", "ec", "CofactorFinal"]]

#cofactor_model.to_csv(os.path.join(DATA_DIR, "cofactors", "cofactor-model-media.csv"), index=False) #SAVE
#cofactor_model.head()

## EC to KO

In [ ]:
from Bio.KEGG import REST
import modules.kegg as kegg

### [5.1] Retrieve KEGG Orthologs 

In [ ]:
taxa_ec = pd.read_csv(os.path.join(DATA_DIR, "taxa2ec-final.csv"), low_memory=False)

# Filter non-sepcific EC numbers (e.g., 1.1.1.-)
filter = taxa_ec["ec"].str.contains("-")
taxa_ec = taxa_ec[~filter]

# Make into a list of unique ECs
ec_list = taxa_ec["ec"].to_list()
ec_list = set(ec_list)
ec_list = list(ec_list)

# Retrieve KOs from KEGG
ec2ko_df = kegg.ec2ko(ec_list)

In [ ]:
# Merge on original dataframe
ko_df = pd.merge(left=taxa_ec, right=ec2ko_df, on="ec", how="left")

ko_df.to_csv(os.path.join(DATA_DIR, 'ec2ko-taxa.csv'), index=False) #SAVE
ko_df.head()

# TODO: use the taxa2ec and media2ec lists as inputs, then use KEGG decoder

## Metabolites

### [6.1] Formatting metabolite information (BacDive)

General metabolite information

In [6]:
bacdive_df = pd.read_csv(os.path.join(DATA_DIR, 'bacdive', 'bacdive-all.csv'), low_memory=False) #Additonal references and other info also available

mtblt = bacdive_df[[
    "bacdive_id",
     "taxon_id",
     "compound_production_compound",
     "metabol_uti",
     "chebi_id",
     "metabol_name",
     "metabol_production", 
     "isolation,_sampling_and_environmental_information_isolation"
]].copy()

mtblt.to_csv(os.path.join(DATA_DIR, "bacdive", "metabolites.csv"), index=False) #SAVE
mtblt.head()

,bacdive_id,taxon_id,compound_production_compound,metabol_uti,chebi_id,metabol_name,metabol_production,"isolation,_sampling_and_environmental_information_isolation"
0,24370,294,trans carveol,"[{'@ref': 68369, 'Chebi-ID': 25115, 'metabolit...",35581.0,indole,no,NaN
1,24368,306,NaN,"[{'@ref': 68371, 'metabolite': 'Potassium 5-ke...",35581.0,indole,no,NaN
2,23995,61646,NaN,NaN,NaN,NaN,NaN,NaN
3,17621,346,restriction endonuclease <i>XmaI</i>,NaN,NaN,NaN,NaN,"[{'@ref': 1479, 'sample type': 'plant (cotton)..."
4,17596,339,restriction endonuclease <i>Xba</i>I,"[{'@ref': 117115, 'Chebi-ID': 16947, 'metaboli...",35581.0,indole,no,"[{'@ref': 1480, 'sample type': '<I>Xanthium st..."


Metabolite Utilization

In [7]:
# Function to parse through metabol_uti information and explode into new columns

def expand_dict_list(df, column):
    # Convert the string representation of the list of dictionaries into actual lists
    df[column] = df[column].apply(ast.literal_eval)
    
    # Explode the column with lists of dictionaries to individual rows
    df_expanded = df.explode(column).reset_index(drop=True)
    
    # Normalize the dictionaries into a flat dataframe
    expanded_rows = pd.json_normalize(df_expanded[column])
    
    # Combine the original dataframe (excluding the original column) with the expanded rows
    result = pd.concat([df_expanded.drop(columns=[column]), expanded_rows], axis=1)
    
    return result

In [8]:
# Format mtblt dataframe to apply function (string type, no na/Nan/nan)
mtblt["metabol_uti"] = mtblt["metabol_uti"].astype(str)
mtblt["metabol_uti"] = mtblt["metabol_uti"].dropna()
filter = mtblt["metabol_uti"].str.contains("nan")
mtblt = mtblt[~filter]

# Apply function
expanded_df = expand_dict_list(mtblt, 'metabol_uti')

# Reapply nan filter for the newly expanded data
expanded_df = expanded_df.copy()
expanded_df["utilization activity"] = expanded_df["utilization activity"].astype(str)
expanded_df["utilization activity"] = expanded_df["utilization activity"].dropna()
filter2 = expanded_df["utilization activity"].str.contains("nan")
expanded_df = expanded_df[~filter2]

# Filter for utilization activity (+)
#filter3 = expanded_df["utilization activity"].str.contains(r"\+")
#expanded_df = expanded_df[filter3]

# Subset and save metabolite utilization information
metabol_uti = expanded_df[[
    "bacdive_id",
    "taxon_id",
    "Chebi-ID",
    "metabolite",
    "utilization activity",
    "kind of utilization tested",
    "isolation,_sampling_and_environmental_information_isolation"
    ]]
metabol_uti = metabol_uti.rename(columns={
    "Chebi-ID": "chebi_id",
    "isolation,_sampling_and_environmental_information_isolation": "isolation_info"
    })

metabol_uti.to_csv(os.path.join(DATA_DIR, "bacdive", "metabol_uti.csv"), index=False) #SAVE
metabol_uti.head()

,bacdive_id,taxon_id,chebi_id,metabolite,utilization activity,kind of utilization tested,isolation_info
0,24370,294,25115.0,malate,+,assimilation,NaN
1,24370,294,17128.0,adipate,-,assimilation,NaN
2,24370,294,27689.0,decanoate,+,assimilation,NaN
3,24370,294,24265.0,gluconate,+,assimilation,NaN
4,24370,294,17306.0,maltose,-,assimilation,NaN


Metabolite Production

In [9]:
mtblt["metabol_production"] = mtblt["metabol_production"].astype(str).copy()

# Subset rows with confirmed metabolite production
substring = 'yes'
filter = mtblt["metabol_production"].str.contains(substring)
metabol_pro = mtblt[filter]

# Subset and save metabolite production information
metabol_pro = metabol_pro[[
    "bacdive_id",
    "taxon_id",
    "chebi_id",
    "metabol_name",
    "metabol_production",
    "isolation,_sampling_and_environmental_information_isolation"
    ]]
metabol_pro = metabol_pro.rename(columns={
    "metabol_name": "metabolite",
    "isolation,_sampling_and_environmental_information_isolation": "isolation_info"
    })

metabol_pro.to_csv(os.path.join(DATA_DIR, "bacdive", "metabol_pro.csv"), index=False) #SAVE
metabol_pro.head()

,bacdive_id,taxon_id,chebi_id,metabolite,metabol_production,isolation_info
63,5030,29488,35581.0,indole,yes,NaN
95,347,75697,35581.0,indole,yes,"[{'@ref': 4618, 'sample type': 'ditch in a for..."
96,346,75697,35581.0,indole,yes,"[{'@ref': 4617, 'sample type': 'activated slud..."
180,5050,1004166,35581.0,indole,yes,"[{'@ref': 16317, 'sample type': '<i>Heterorhab..."
181,5036,2218628,35581.0,indole,yes,"[{'@ref': 5632, 'sample type': 'nematode <I>He..."


## Enviromental Parameters

In [3]:
# Subsetting MediaDive dataframe
md_df = pd.read_csv(os.path.join(DATA_DIR,"taxa2ec-explode.csv"), low_memory=False)

# Format pH data as a range
def create_range(row):
    if row['min_pH'] == row['max_pH']:
        return f"{row['min_pH']}"
    else:
        return f"{row['min_pH']}-{row['max_pH']}"
md_df['pH'] = md_df.apply(create_range, axis=1)
md_df = md_df[["media_id", "name", "pH","components", "component_ids", "species", "ccno", "bacdive_id", "ec_bacdive","ec_uniprot","ec_ncbi"]]

# BacDive dataframe
bd_df = pd.read_csv(os.path.join(DATA_DIR, "bacdive", "bacdive-all.csv"), low_memory=False)

# Merge dataframes
dive = pd.merge(left=md_df, right=bd_df, on="bacdive_id", how="left")
dive.head()

,media_id,name,pH,components,component_ids,species_x,ccno,bacdive_id,ec_bacdive,ec_uniprot,...,api_id32sta_beta_gur,physiology_and_metabolism_murein,physiology_and_metabolism_api_list,isolation_enrichment_culture,isolation_enrichment_culture_temperature,multicellular_morphology_complex_color,api_list_beta_hem,metabolite_tests_citrate_test,compound_production_excreted,isolation_enrichment_culture_duration
0,1,NUTRIENT AGAR,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1a,REACTIVATION WITH LIQUID MEDIUM 1,7.0,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",Comamonas testosteroni,DSM 38,2912.0,"[1.9.3.1, 3.2.1.21, 3.5.1.5, 3.5.3.6]","[2.6.1.1, 4.1.1.12, 1.13.11.74, 1.13.11.76, 1....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1a,REACTIVATION WITH LIQUID MEDIUM 1,7.0,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",Delftia acidovorans,DSM 39,2941.0,"[1.9.3.1, 3.2.1.21, 3.5.1.5, 3.5.3.6]","[1.14.11.43, 1.14.11.44, 3.1.1.75, 6.3.4.2, 5....",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1a,REACTIVATION WITH LIQUID MEDIUM 1,7.0,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",Acidovorax delafieldii,DSM 64,2885.0,"[3.2.1.51, 3.2.1.24, 3.2.1.52, 3.2.1.21, 3.2.1...",[1.1.1.37],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1a,REACTIVATION WITH LIQUID MEDIUM 1,7.0,"['Peptone', 'Meat extract', 'Agar', 'Distilled...","[1, 2, 3, 4]",Pseudomonas putida,DSM 84,12895.0,"[3.2.1.21, 3.5.1.5, 3.5.3.6]","[5.1.1.10, 5.3.3.1, 1.18.1.3, 1.2.98.1, 1.18.1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concentration Mapping

In [8]:
# Load dataframe with CoCl2 media concentrations
cocl = pd.read_csv(os.path.join(DATA_DIR, "concentrations", "CoCl2_trimmed.csv"))

# Load dataframe with EC annotations
ec = pd.read_csv(os.path.join(DATA_DIR, "taxa2ec-final.csv"), low_memory=False)

# Merge these dataframes to associate ec values w/ concentrations
coan = pd.merge(left=cocl, right=ec, on="media_id", how="left")
coec = coan.dropna(subset=["ec"])
filter = coec["ec"].str.contains("-")
coec = coec[~filter]

coec.head()

,solution,media_id,CoCl2,species,taxon_id,source,ec
0,Media,150a,0.6,Acidithiobacillus caldus,"[{'NCBI tax id': 33059, 'Matching level': 'spe...",uniprot,2.4.1.13
13,Allen,88a,0.0,Vulcanisaeta distributa,"[{'NCBI tax id': 164451, 'Matching level': 'sp...",uniprot,2.5.1.157
14,Allen,88a,0.0,Vulcanisaeta distributa,"[{'NCBI tax id': 164451, 'Matching level': 'sp...",uniprot,5.4.99.25
21,Allen,358a,0.0,Acidianus ambivalens,2283,uniprot,1.8.5.2
22,Allen,358a,0.0,Acidianus ambivalens,2283,uniprot,1.8.5.2


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

df = coec.copy()
ec_range = df.groupby('ec')['CoCl2'].agg(['min', 'max']).reset_index()
ec_range.columns = ['EC Number', 'Min Concentration', 'Max Concentration']

# Initialize
fig = go.Figure()

# Error bars
fig.add_trace(go.Scatter(
    x=ec_range['EC Number'],
    y=(ec_range['Max Concentration'] + ec_range['Min Concentration']) / 2,  # Mean concentration for y-axis
    error_y=dict(
        type='data',
        symmetric=False,
        array=(ec_range['Max Concentration'] - ec_range['Min Concentration']) / 2,
        arrayminus=(ec_range['Max Concentration'] - ec_range['Min Concentration']) / 2
    ),
    mode='markers',
    marker=dict(color='blue'),
    text=ec_range['EC Number'],  # Hover text
    hoverinfo='text',  # Show hover text
    name='Concentration Range'
))

# Update layout
fig.update_layout(
    title='Concentration Range for Each EC Number',
    xaxis_title='EC Number',
    yaxis_title='Concentration',
    xaxis_tickangle=-45
)

fig.show()